# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [75]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201108141815.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [76]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [79]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from lendingPoolCore.Math import *

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




In [80]:
pprint(contracts)

{'addressProvider': 'cxed440f1da2cf3087ac5ef12217a13f6321de0375',
 'feeProvider': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
 'lendingPool': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'lendingPoolCore': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'lendingPoolDataProvider': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
 'oICX': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'oToken': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
 'priceOracle': 'cx3f6477bf290eb91257b806ece614ab18b1c7be41',
 'sample_token': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}


### Required service

In [81]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [5]:


"""
contracts = {

}
pprint(contracts)
"""
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}


# Importing wallet

### Importing from private key

In [82]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [7]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78700095085177400000000000

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

In [7]:
private2="b3d4e18d733f05dc21f326ba4fe150514874327c738c4a19ddd357d0e8e2efed"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="c3d5e9afe753a07c983bb14e34f73907e40a997793ce2f7d2895cddec27f81c6"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

private4="2ffb9ef51826d555189d4804161058036cf87ae8290f1a308a79d045f6cdd428"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())
wallets=[deployer_wallet2,deployer_wallet3,deployer_wallet4]

hxd29037f1eb8c3f0e8630d797edc7410507c0089a
hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
hx65902dd4139d04765534a4539319be874ad7aa0b


# Deploying all contracts


In [9]:
#contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'liquidationManager']
# for directory in {"score"}:
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[item] = res.get('scoreAddress', '')

#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending tra

{'addressProvider': 'cxc595228d12cc267c748026a6629af41060654dab',
 'feeProvider': 'cxb76322c7df895a14e47bc14b4aa21566b3edc069',
 'lendingPool': 'cxf44d5c379d7b458fe4bba1ecb0f8fd4a54e9d65c',
 'lendingPoolCore': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'lendingPoolDataProvider': 'cxfdff702158ae5bfce3f25a4c040b850d96e25af8',
 'oICX': 'cxf872138faf25dfc5a1216a7a85cad24b8172d988',
 'oToken': 'cx9732efc2d75905dfe4d8bc2d055c2cf226391d00',
 'priceOracle': 'cx1fbce991b255545bcd31634ab6beda5605201f46',
 'sample_token': 'cxc88281539407ee6d766e58d302f27c50d517783c',
 'sicx': 'cx400639d22b6bce4def6e7d7be2bc36e7a39192ee',
 'liquidationManager': 'cx847bb94d386b34af9ba86dfdcace651dbd5af822'}

In [13]:
now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

saving contracts list as contracts_20201108141815.pkl


## Save updated contract

In [46]:
#use this to update a pre existing pkl file
filename = 'updated20201125_' + contracts_file_custom
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as updated20201125_custom_contracts_20201122161857.pkl


{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx6703fdc7e0e65c5195d6dbfb59a838f76a9c19b1'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

# ICX transfer

In [17]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae")\
    .nid(3) \
    .nonce(100) \
    .value(1000000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x3c546494f559d4ea9d34534f13387c74aa80658c67ac2630ce1adaf514e1bc86',
 'blockHeight': 1425621,
 'blockHash': '0xc6a2b22340f3c2756027f1926f731d1eaa78ed6c65653e92e85f58e7ee2d2b66',
 'txIndex': 1,
 'to': 'hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [25]:
params = {"_owner":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("balanceOf")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x2e044c431b4da18bd8d57bb81662e176a9e0c35d0258d0f8fdd9e064801d6738',
 'blockHeight': 10244800,
 'blockHash': '0x136d86ac10f8ae06e8b9c76f13b9074e34011902bbfe20314ac21d4215a77af9',
 'txIndex': 1,
 'to': 'cx93119bb51b7885b502293882004bbc5dc74e8215',
 'stepUsed': 142400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 142400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Updating a contract

In [88]:
#deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'oICX', 'liquidationManager']
#for contract_ in deploy:


contract_ = 'oICX'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


update oToken cxf6e3d64db674c0edbf732462e80e2e3352280548
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8b8ba012c9f3d7b9a10b76be01b90842caab62c091a914672c418f33f11d5103',
 'blockHeight': 11575540,
 'blockHash': '0x5a0fe4ff5c60c6d3c16a2a5bf88c44d9fbcbb3c66cc9ad3806a3d025ddee8c66',
 'txIndex': 1,
 'to': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'scoreAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'stepUsed': 1743719200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1743719200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [66]:
params = {"_reserveAddress": contracts['sicx'], "_liquidationBonus": 1 * 10 ** 17 }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("updateLiquidationBonus")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x2b9631e6573e89fdaa9c672b46fea43fc2453f26e641d564f9c0dab5e4e75234',
 'blockHeight': 1787871,
 'blockHash': '0x5d5730cfdc2ece16aa225eaaf85d7a12f6d57643f20882ac4249e489d7a5c594',
 'txIndex': 1,
 'to': 'cxc34e4610de51d36edddcb8822e393dcbb14603a4',
 'stepUsed': 155440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 155440,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [15]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress','params': {'_address':contracts['liquidationManager']}},
            {'contract': 'lendingPool', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'lendingPool', 'method': 'setOICXAddress', 'params':{'_address': contracts['oICX']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"ICX"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
            {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},  
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
            {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
            {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'liquidationManager', 'method': 'setOracleAddress','params': {'_address':contracts['priceOracle']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},]

In [16]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

'stepUsed': 149120, 'stepPrice': 10000000000, 'cumulativeStepUsed': 149120, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding USDb reserve in LendingPoolCore



In [17]:
params ={"_reserve": {"reserveAddress":contracts['sample_token'],"oTokenAddress":contracts['oToken'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x60ce824a361933578553b94aa4267d2435c07e3681f164e72fbcdfc42b220a27',
 'blockHeight': 1870957,
 'blockHash': '0x5148eb18e52af3de67f5ceeccc02cb725b962a68b824df9db938aee4d77883bd',
 'txIndex': 1,
 'to': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'stepUsed': 277040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 277040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding ICX reserve

In [18]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x62d984b4d8378d1e0cf0c2067ab7ed32da82f111b4cb2755586879824c6434ef',
 'blockHeight': 1870960,
 'blockHash': '0xa2b37b48ba8f5b076c482ec35d6f822e1c053f20121632ee1a4ce947b73f65a1',
 'txIndex': 1,
 'to': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transfering USDb to a wallet

In [25]:
params = {"_to": "hxb6084c615cc424ef2fc5329c07e2e6d2fbc7f0f8", "_value": 10000 * 10 ** 18}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc07b16c637e63952a12a3b781feb0850c60c1bbf9a749d471d9dc64d8d4725a9',
 'blockHeight': 11297686,
 'blockHash': '0xec3e2eb18730a98e4388d0efe781d08ba83163bc4a56c9fa83723f63a4d94685',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 161460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 161460,
 'eventLogs': [{'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hxb6084c615cc424ef2fc5329c07e2e6d2fbc7f0f8',
    '0x21e19e0c9bab2400000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x02\x00\x00\x00\x00\

In [8]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cxed440f1da2cf3087ac5ef12217a13f6321de0375',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'oICX': {'SCORE': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
          'zip': '/home/shubham/Work/ICON/openmoneymarket-mo

# Depositing USDb from a wallet

In [109]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0xd61dd328e5dfa251632609893233dd383a15c76e680cf0b5f4d8751867e6340a',
 'blockHeight': 1353677,
 'blockHash': '0xa0b52c7c33c17a11f4d114baab6c9a18c87e500fd8e375cdaa00395c2b8aba9c',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 751720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 751720,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0xde0b6b3a7640000', '0xd

# Borrowing USDb

In [110]:
params ={"_reserve": contracts['sample_token'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1', 'stepLimit': '0x146dc4', 'timestamp': '0x5b553bdb121c6', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d', '_amount': '0x8ac7230489e80000'}}, 'signature': 'yxSWWut7ryf4/6NtDopmBxaHiCw+r5dhQzzezt7x14YKG0fSVISEjPYE6Timllqvv4lfWcJ4RD+A7/ww7J3/5gE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xd14ec29c4c2ecb85012d82d6882410d9399c4a41da0a560a472262781a009594',
 'blockHeight': 1353685,
 'blockHash': '0x0ccccb53393581722268081256f49cd12598f808c55a12c9189e73283682de1b',
 'txIndex': 1,
 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
 'stepUsed': 1238820,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1238820,
 'eventLogs': [{'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0xe74be5079082aaab',
    '0x0'],
   'data': ['0x0']},
  {'scoreAdd

# Reedeming/Withdrawing USDb

In [111]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65', 'stepLimit': '0x143ebc', 'timestamp': '0x5b553bf01f180', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'redeem', 'params': {'_amount': '0x8ac7230489e80000'}}, 'signature': 'A6J8pKu1O6z4b6+yRLpfqwyJ4duW7yxDUyS/z0tw5TRaYflRZ4z98yS881ACeFNCL+V/TjvTpJGOWm35iNIKoQE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5b0cc66a6ff064bf2ecc8d9d50b0512570308396c11a958925a403a1ec357a5f',
 'blockHeight': 1353696,
 'blockHash': '0x2a41466d89cd9b869a0500d849a6422bf8e41065b272106100973ce157886430',
 'txIndex': 1,
 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
 'stepUsed': 1226780,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1226780,
 'eventLogs': [{'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xf97c99928']},
  {'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x8ac7230489e80000']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0x8ac7230489e80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed

In [112]:
params ={"_reserveAddress": contracts['sample_token'],"_baseLTVasCollateral" :33*10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855', 'stepLimit': '0x3ea80', 'timestamp': '0x5b553c06ebdc6', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d', '_baseLTVasCollateral': '0x494654067e10000'}}, 'signature': '79de8WIdF8kiL8fPiEdEOhCZE8E3ccr1MEVFQLQv1G8+mWWe7G22aAzjG93wIm6O9OG3xu1QOnEF0vcSfNApbAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x9b656a472068de6e8f2b40f3003b4cfcae8fb206557ecef3a00da84c67d816c7',
 'blockHeight': 1353708,
 'blockHash': '0xd794c262f864c4f681a19ed983b8b36cb5f855cdaf0244ecc6ab7b1cace2fcef',
 'txIndex': 1,
 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 156640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Repaying USDb

In [113]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x277fae145ad2b5aba3b4658c75f9c795b9cd26d2a014e803f440349f706b0ed5',
 'blockHeight': 1353714,
 'blockHash': '0x807be102e3e53321d0192a4fea9efc26acedd859277c2fcfe785651ebea33f5b',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 929860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 929860,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x8a6e51a672858000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0xde0b6b

In [23]:
print(icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"))

9450000385630198414715


# Depositing ICX from Wallet

In [10]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}


In [35]:
params = {"_amount": 50 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(50 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x41a41b24672c9d75a0e4a0b929ff8e1237476eac6ea3f695a867687f72b7a74a',
 'blockHeight': 11571320,
 'blockHash': '0xb2fb5cb050d4c286955b1ee0a5996f973445d6530f5f2d27ee95316c8865339a',
 'txIndex': 1,
 'to': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'stepUsed': 893020,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 893020,
 'eventLogs': [{'scoreAddress': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
    'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
    '0x2b5e3af16b1880000'],
   'data': []},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
    'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
    '0x2b57aaf7f533de0d2'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Mint(Address,int,bytes)',
    'cx

# Withdrawing ICX 

In [48]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['staking'])\
    .method("getTodayRate")\
    .build()
todayRate = int(icon_service.call(_call),0)
pprint(todayRate)
# ICX_amount = 10 * ICX
# sicx_amount =exaDiv(ICX_amount,todayRate)
# print(sicx_amount)

1000591438055510743
9994089115367006256


In [73]:
ICX_amount = 20 * ICX
sicx_amount =exaDiv(ICX_amount,todayRate)
params = {"_amount": sicx_amount,"_waitForUnstaking":1}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
        

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa954018acdf12a077db10a19c1ba9d1915538d4c8626d9da88c024e16dc0f247',
 'blockHeight': 11573867,
 'blockHash': '0xe688be7bbf0b89dbe349ff6c4fe424e7a74097cab73ec61fb03fb2e376745e8e',
 'txIndex': 1,
 'to': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'stepUsed': 1351100,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1351100,
 'eventLogs': [{'scoreAddress': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x115644632ee18c060']},
  {'scoreAddress': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0x115644632ee18c060'],
   'data': ['0x0']},
  {'scoreAddress': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
   'indexed': ['

In [52]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}


# Borrowing sicx

In [26]:
#borrow sicx
params ={"_reserve": contracts['sicx'], "_amount":100*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554', 'stepLimit': '0x18a290', 'timestamp': '0x5b56a6cdd75f1', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27', '_amount': '0x56bc75e2d63100000'}}, 'signature': 'FpOevHks4R+7T08A8dEF5XIXJgZljUnkjYvuFmke3ytj0fJBOrAu+BrUzZrMTFaKN2P38jH5WgrgHR7N0jnHywE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}
{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x1f0f72eab1db758de1b644b7e1054de762de98699c7e2f5c4f840d01cc6fb92a',
 'blockHeight': 9908027,
 'blockHash': '0xd3b91a1a3a7cf36f636c3a0b57a6f6e7ac032add02a83002308a1415f7524cbd',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 1514480,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1514480,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0xba4414334ab230000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x14c09b0f07ec6d8a819',
    '0x1174b6c563054415d'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0xb850adebe4f559f48',
    '0x0'],
   'data': ['0x0

# Repay sicx

In [29]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4b95d8683af8ba72d8ab4ba223c31b1c9c2f5e5b9ad262a4fa6f1a2ad45138f7',
 'blockHeight': 9908080,
 'blockHash': '0x51f885287ac717250a1775e9dc72600273acde6d78f3b4689fff9b3a40594a29',
 'txIndex': 1,
 'to': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
 'stepUsed': 867000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 867000,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x5684f308079370000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0

# Readonly methods to get data

### Getting all the addresses

In [14]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'USDb': 'cx9da823297393ecb193030b983ce334f25af20aac',
                'sICX': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'},
 'oTokens': {'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
             'oUSDb': 'cx77d34d57f7ea300637058ef65babc52da91376a4'},
 'systemContract': {'LendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
                    'LendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582'}}


### Getting specific reserve data

In [53]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 201.01049207728127,
 'baseLTVasCollateral': 0.5,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'exchangePrice': 0.5,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1610447043554731,
 'liquidationBonus': 0.1,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'reserveAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'todaySicxRate': 1.0005914380555108,
 'totalBorrows': '0x0',
 'totalLiquidity': 201.01049207728127,
 'totalSicxBalance': 200.89167709441227,
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [54]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 100.50524603864064,
 'totalBorrowsBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 100.50524603864064,
 'totalLiquidityBalanceUSD': 100.50524603864064}


### Getting data of all reserves

In [55]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'ICX': {'availableLiquidity': 201.01049207728127,
         'baseLTVasCollateral': 0.5,
         'borrowCumulativeIndex': 1.0,
         'borrowRate': '0x0',
         'borrowingEnabled': '0x1',
         'decimals': 1.8e-17,
         'exchangePrice': 0.5,
         'isActive': '0x1',
         'isFreezed': '0x0',
         'lastUpdateTimestamp': 1610447043554731,
         'liquidationBonus': 0.1,
         'liquidationThreshold': 0.65,
         'liquidityCumulativeIndex': 1.0,
         'liquidityRate': '0x0',
         'oTokenAddress': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
         'reserveAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
         'todaySicxRate': 1.0005914380555108,
         'totalBorrows': '0x0',
         'totalLiquidity': 201.01049207728127,
         'totalSicxBalance': 200.89167709441227,
         'usageAsCollateralEnabled': '0x1'},
 'USDb': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': 0.5,
          'borrowCumulativeIndex': 1.0,
          

### Getting user account data

In [56]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'availableBorrowsUSD': 50.25262301932032,
 'borrowingPower': 0.0,
 'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.5,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 100.50524603864064,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 100.50524603864064}


In [49]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [57]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 201.01049207728127,
 'currentOTokenBalanceUSD': 100.56468866592635,
 'exchangeRate': 0.5002957190277554,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'principalOTokenBalance': 201.01049207728127,
 'principalOTokenBalanceUSD': 100.56468866592635,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 0.0}


# Getting All Reserve Data

In [44]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'USDb': {'availableLiquidity': '0x3f35b7a91f3815900000',
          'baseLTVasCollateral': '0x853a0d2313c0000',
          'borrowCumulativeIndex': '0xde0b7c160e6fb0d',
          'borrowRate': '0x128fc221d15a6e5',
          'borrowingEnabled': '0x1',
          'decimals': '0x12',
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x5b4bfa2ee7bf9',
          'liquidationBonus': '0xa',
          'liquidationThreshold': '0x905438e60010000',
          'liquidityCumulativeIndex': '0xde0b73a80e1fe23',
          'liquidityRate': '0xa78e20466e3ea9',
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': '0x6a32602b7ababa00e840',
          'totalLiquidity': '0xa96817d499f2cf90e840',
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': '0x494654067e10000',
          'borrowCu

### Getting the data of user for all reserves

In [42]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0},
 'sICX': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


### Get reserve configurations


In [61]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [62]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'icx': {'baseLTVasCollateral': 330000000000000000,
         'borrowingEnabled': 1,
         'decimals': 18,
         'isActive': 1,
         'liquidationBonus': 10,
         'liquidationThreshold': 650000000000000000,
         'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd
